In [2]:
using Symbolics
using Plots

#custom dependencies
include("derivatives.jl")
include("interpolations.jl")

general_richardson (generic function with 1 method)

In [ ]:
function f1(x)
    return tan(x/3)
end

function f2(x)
    return eps(x) + x
end

Derivate both functions using three and five points formula, with middle point cause is more accuracy